In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import pandas as pd

from sklearn.metrics import accuracy_score

import time

import os
import glob, time
import io
import random
import re

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving UNSWdata.tar.gz to UNSWdata.tar.gz
User uploaded file "UNSWdata.tar.gz" with length 122706982 bytes


In [0]:
'''
UNSWdata.tar.gz contains 4 tar.gz files: UNSW-NB15_*.tar.gz where * is 1 through 4.
Each UNSW-NB15_*.tar.gz contains a corresponding .csv file.
This was necessary because the original files are roughly 750MB, so loading directly
from .csv into a dataframe takes about an hour. With the tar.gz compression, this process
is cut down to about 5 minutes.
'''

# Extract from master tar.gz file
import tarfile
tf = tarfile.open(name='UNSWdata.tar.gz')
tf.extractall()

In [0]:
# Extract csv files from each tar.gz file
for i in range(1,5):
  tf = tarfile.open(name='UNSW-NB15_'+str(i)+'.tar.gz')
  tf.extractall()

In [0]:
# Read csv files into one dataframe

temp_list = []

for i in range(1,5):
    filename = 'UNSW-NB15_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=None, header=None)
    temp_list.append(df)

data = pd.concat(temp_list, axis=0, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
from google.colab import files
start = time.time()
uploaded = files.upload()
end = time.time()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
print(end-start)

Saving UNSW-NB15_features.csv to UNSW-NB15_features.csv
User uploaded file "UNSW-NB15_features.csv" with length 4044 bytes
6.180320978164673


In [0]:
# getting feature names

feature_list = pd.read_csv('UNSW-NB15_features.csv',encoding='unicode_escape')
column_names = list(feature_list.Name)
data.columns = column_names

In [0]:
data.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0


In [0]:
data.shape

(2540047, 49)

In [0]:
print('Number of total attacks:',data.Label.sum(),'out of 2540047')
print('Percent attacks:',round(data.Label.mean(),2)*100,'%')

Number of total attacks: 321283 out of 2540047
Percent attacks: 13.0 %
